In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow  # Import cv2_imshow for Colab

# 画像ファイルのパス
image_path = '/content/スクリーンショット 2024-11-26 134910.png'

# 画像を読み込む
image = cv2.imread(image_path)
if image is None:
    raise ValueError("画像が読み込めませんでした。ファイルパスを確認してください。")

# 色空間をBGRからHSVに変換
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 青色の閾値を設定（盤面の端を検出）
lower_blue = np.array([100, 150, 50])
upper_blue = np.array([140, 255, 255])

# 青色領域のマスクを作成
mask_blue = cv2.inRange(hsv_image, lower_blue, upper_blue)

# マスクを基に輪郭を検出
contours, _ = cv2.findContours(mask_blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 最大の輪郭（盤面の外枠）を特定
if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    # 盤面を切り出し（さらに内部の領域を切り取るために余白を削除）
    margin = 10  # 余白のサイズ
    x, y, w, h = x + margin, y + margin, w - 2 * margin, h - 2 * margin
    board_image = image[y:y+h, x:x+w]
else:
    raise ValueError("盤面の外枠が検出できませんでした。青色の範囲を確認してください。")

# 切り出した盤面をリサイズ（7×6のマス目に合わせる）
resized_image = cv2.resize(board_image, (700, 600))

# 色空間をHSVに変換
hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)

# 赤色と黄色の閾値を設定
lower_red1 = np.array([0, 100, 50])
upper_red1 = np.array([15, 255, 255])
lower_red2 = np.array([165, 100, 50])
upper_red2 = np.array([180, 255, 255])
lower_yellow = np.array([15, 80, 80])
upper_yellow = np.array([35, 255, 255])

# 赤色と黄色のマスク
mask_red1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
mask_red2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
mask_red = cv2.bitwise_or(mask_red1, mask_red2)
mask_yellow = cv2.inRange(hsv_image, lower_yellow, upper_yellow)

# ボードのマスを縦6、横7のグリッドに分割
grid_rows, grid_cols = 6, 7
cell_height = resized_image.shape[0] // grid_rows
cell_width = resized_image.shape[1] // grid_cols

# 結果画像と出力フォーマット
result_image = resized_image.copy()
board_state = []

# 判定の処理
for row in range(grid_rows):
    row_state = []
    for col in range(grid_cols):
        # 各セルの範囲を計算
        y_start, y_end = row * cell_height, (row + 1) * cell_height
        x_start, x_end = col * cell_width, (col + 1) * cell_width

        # 各マスクのピクセル値を集計
        red_density = np.mean(mask_red[y_start:y_end, x_start:x_end])
        yellow_density = np.mean(mask_yellow[y_start:y_end, x_start:x_end])

        # 色の割合で判定
        if red_density > yellow_density:
            row_state.append('R')
            result = 'R'
            color = (0, 0, 255)  # 赤色
        elif yellow_density > red_density:
            row_state.append('Y')
            result = 'Y'
            color = (0, 255, 255)  # 黄色
        else:
            row_state.append('-')
            result = '-'
            color = (255, 255, 255)  # 白色

        # 結果を描画
        cv2.putText(result_image, result, (x_start + 10, y_start + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        # マス目の境界線を描画
        cv2.rectangle(result_image, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2)
    board_state.append(row_state)

# 結果をフォーマットして表示
output = "\n".join(" ".join(row) for row in board_state)
print(output)

# 画像を表示
cv2_imshow(result_image)
